# <FONT COLOR=BLUE>FOOD DESERTS IN NORFOLK, VA</FONT>

## INTRODUCTION
### Norfolk, Virginia
The following notebook is designed to explore food deserts in Norfolk, Virginia. Norfolk is a port city in southeast Virginia. It is the second most populous city, and the fourth most densely populated, in the state.

### What is a food desert?
The USDA defines a food desert as a low-income census tract where a substantial number or share of residents has low access to a supermarket or large grocery store. 

Only census tracts listed on the Treasury Department's New Markets Tax Credit (NMTC) qualify as low-income. 

Low access to a supermarket or large grocery store is defined in urban areas as a radius of more than one mile, and in rural areas as more than 10 miles. All of Norfolk is considered urban.

If at least 33% or 500 people, whichever is less, in the NMTC tract have low access it is considered a food desert.

### Who is interested in this information?
Understanding the under-served communities within an area is of interest to both government planners and business leaders alike. To government planners because it helps them to identify the areas that need assistance and growth, and to business leaders because it helps them identify areas for expansion.

## DATA

For this project I will be using the following data:

Census tract geographical information for the Norfolk, VA. This was available from hrgeo.org which is a geospatial exchange for Hampton Roads. Hampton Roads is the geographical region which includes Norfolk.

NMTC identified low-income areas. This data was available from cdfifund.gov. 

Residential addresses in Norfolk. This was available from data.norfolk.gov.

Grocery store locations. foursquare.com api will be used to gather this info. I will use the center point of the census tract as the center point for the foursquare search.

## METHODOLOGY

Import necessary libraries

In [1]:
#import libraries

import pandas as pd #for data analysis
import numpy as np #import numpy

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#Import necessary libraries and handlers
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.11.0 --yes 
import folium # map rendering library

!pip install geojson
from geojson import Polygon

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

Retrieve data.

In [2]:
tracts = f'./NorfolkCensusTracts.json'
hr_NMTCTracts = pd.read_csv("2019 NMTC Public Data Release_FY_17_HR.csv")
norf_NMTCTracts = hr_NMTCTracts[hr_NMTCTracts['City']=='Norfolk']

In [3]:
norf_NMTCTracts.head(9)

,Project ID,2010 Census Tract,"Metro/Non-Metro, 2010 Census",City,Standard_City,State,Standard_State,Zip Code
3,VA0019,51710004200,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
4,VA0021,51710004700,Metropolitan,Norfolk,Norfolk,VA,Virginia,23504
5,VA0022,51710004400,Metropolitan,Norfolk,Norfolk,VA,Virginia,23504
6,VA0023,51710004900,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
7,VA0024,51710004002,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
8,VA0025,51710002600,Metropolitan,Norfolk,Norfolk,VA,Virginia,23508
10,VA0027,51710980300,Metropolitan,Norfolk,Norfolk,VA,Virginia,23502
11,VA0028,51710006604,Metropolitan,Norfolk,Norfolk,VA,Virginia,23518
19,VA0083,51710002900,Metropolitan,Norfolk,Norfolk,VA,Virginia,23517


Find coordinates for Norfolk, VA.

In [4]:
address = 'Norfolk, VA'

geolocator = Nominatim(user_agent="norfolk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Norfolk, Virginia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Norfolk, Virginia are 36.8462923, -76.2929252.


In [5]:
# create map of Norfolk using latitude and longitude values
map_hr = folium.Map(location=[latitude, longitude], zoom_start=11)

The map below shows <b>all</b> of the Census Tracts for Norfolk from the 2010 census. We will only be interested in the NMTC tracts.

In [6]:
folium.GeoJson(
    tracts,
    name='tracts',
    #style_function=style_function,
    #object_path='objects.features'
).add_to(map_hr)
map_hr       

Below are the tracts defined by the Treasury Department as low-income tracts.

In [7]:
norf_NMTCTracts.head(9)

,Project ID,2010 Census Tract,"Metro/Non-Metro, 2010 Census",City,Standard_City,State,Standard_State,Zip Code
3,VA0019,51710004200,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
4,VA0021,51710004700,Metropolitan,Norfolk,Norfolk,VA,Virginia,23504
5,VA0022,51710004400,Metropolitan,Norfolk,Norfolk,VA,Virginia,23504
6,VA0023,51710004900,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
7,VA0024,51710004002,Metropolitan,Norfolk,Norfolk,VA,Virginia,23510
8,VA0025,51710002600,Metropolitan,Norfolk,Norfolk,VA,Virginia,23508
10,VA0027,51710980300,Metropolitan,Norfolk,Norfolk,VA,Virginia,23502
11,VA0028,51710006604,Metropolitan,Norfolk,Norfolk,VA,Virginia,23518
19,VA0083,51710002900,Metropolitan,Norfolk,Norfolk,VA,Virginia,23517


Data cleanup

In [8]:
norf_NMTCTracts.drop(['Project ID','Metro/Non-Metro, 2010 Census','City','Standard_City','State','Standard_State','Zip Code'],axis=1,inplace=True)

norf_NMTCTracts.reset_index(drop=True,inplace=True)
norf_NMTCTracts.head(9)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,2010 Census Tract
0,51710004200
1,51710004700
2,51710004400
3,51710004900
4,51710004002
5,51710002600
6,51710980300
7,51710006604
8,51710002900


Now we get the polygons for the specific tracts that we want to focus on.

In [9]:
with open("NorfolkCensusTracts.json", "r") as read_it: 
     geo_data = json.load(read_it) 


In [10]:
norf_tracts = geo_data['features']

In [11]:
## define the dataframe columns
column_names = ['2010 Census Tract', 'Type', 'Coordinates'] 

# instantiate the dataframe
tract_data = pd.DataFrame(columns=column_names)

for data in norf_tracts:
    ctract = data['properties']['GEOID10'] 
    types   = data['geometry']['type']
    coords = data['geometry']['coordinates']
#    neighborhood_name = data['properties']['name']
        
#    neighborhood_latlon = data['geometry']['coordinates']
#    neighborhood_lat = neighborhood_latlon[1]
#    neighborhood_lon = neighborhood_latlon[0]
    
    tract_data = tract_data.append({'2010 Census Tract': ctract,
                                   'Type': types,
                                   'Coordinates': coords}, 
                                   ignore_index=True)

In [12]:
tract_data.head()

,2010 Census Tract,Type,Coordinates
0,51710004001,Polygon,"[[[-76.29418400134314, 36.863587001075665], [-..."
1,51710004002,Polygon,"[[[-76.29895000118125, 36.85478900082314], [-7..."
2,51710004100,Polygon,"[[[-76.28716100127572, 36.85852500128839], [-7..."
3,51710004200,Polygon,"[[[-76.27269200103963, 36.85258500074999], [-7..."
4,51710004300,Polygon,"[[[-76.2731200011328, 36.85150800125312], [-76..."


In [13]:
norf_NMTCTracts=norf_NMTCTracts.astype('object')

In [14]:
norf_data=geo_data['features']

In [15]:
norf_geo = geo_data['features']
norf_NMTCTracts['Type']=""
norf_NMTCTracts['Coordinates']=""
for data in norf_geo:
    tract = data['properties']['GEOID10']
    tract = int(tract)
    stype = data['geometry']['type']
    coords = data['geometry']['coordinates']
    for i in range(1,10):
        ind=i-1
        if(norf_NMTCTracts['2010 Census Tract'][ind]==tract):
            norf_NMTCTracts['Type'][ind]=stype,
            norf_NMTCTracts['Coordinates'][ind]=coords


In [16]:
norf_NMTCTracts['Type']="Polygon"
norf_NMTCTracts.head(9)

,2010 Census Tract,Type,Coordinates
0,51710004200,Polygon,"[[[-76.27269200103963, 36.85258500074999], [-7..."
1,51710004700,Polygon,"[[[-76.2731200011328, 36.85150800125312], [-76..."
2,51710004400,Polygon,"[[[-76.2535640016033, 36.86002200104431], [-76..."
3,51710004900,Polygon,"[[[-76.29898300059763, 36.85473200050688], [-7..."
4,51710004002,Polygon,"[[[-76.29895000118125, 36.85478900082314], [-7..."
5,51710002600,Polygon,"[[[-76.29346700098569, 36.88614800123863], [-7..."
6,51710980300,Polygon,"[[[-76.1960990011138, 36.91140100072901], [-76..."
7,51710006604,Polygon,"[[[-76.20035100149677, 36.92685600061475], [-7..."
8,51710002900,Polygon,"[[[-76.27503800107615, 36.88139700062664], [-7..."


The national census tracts numbers include state and city information. Since the address data for the tracts only include the locality number, we need to add a column that indicates that information. I'm sure there is a better way to do this, but this was effective for a small dataset.

In [17]:
norf_NMTCTracts.insert(2, 'Local CT', "")
norf_NMTCTracts['Local CT']={'42','47','44','49','40.02','26','03','66.04','29'}
norf_NMTCTracts['Local CT'][0]='42'
norf_NMTCTracts['Local CT'][1]='47'
norf_NMTCTracts['Local CT'][2]='44'
norf_NMTCTracts['Local CT'][3]='49'
norf_NMTCTracts['Local CT'][4]='40.02'
norf_NMTCTracts['Local CT'][5]='26'
norf_NMTCTracts['Local CT'][6]='03'
norf_NMTCTracts['Local CT'][7]='66.04'
norf_NMTCTracts['Local CT'][8]='29'
norf_NMTCTracts.head(9)

,2010 Census Tract,Type,Local CT,Coordinates
0,51710004200,Polygon,42,"[[[-76.27269200103963, 36.85258500074999], [-7..."
1,51710004700,Polygon,47,"[[[-76.2731200011328, 36.85150800125312], [-76..."
2,51710004400,Polygon,44,"[[[-76.2535640016033, 36.86002200104431], [-76..."
3,51710004900,Polygon,49,"[[[-76.29898300059763, 36.85473200050688], [-7..."
4,51710004002,Polygon,40.02,"[[[-76.29895000118125, 36.85478900082314], [-7..."
5,51710002600,Polygon,26,"[[[-76.29346700098569, 36.88614800123863], [-7..."
6,51710980300,Polygon,03,"[[[-76.1960990011138, 36.91140100072901], [-76..."
7,51710006604,Polygon,66.04,"[[[-76.20035100149677, 36.92685600061475], [-7..."
8,51710002900,Polygon,29,"[[[-76.27503800107615, 36.88139700062664], [-7..."


Create a new display map for layering the data.

In [18]:
map_norf = folium.Map(location=[latitude, longitude], zoom_start=11)

The map below represents the census tracts in Norfolk that are registered as low income on the Treasure Department's NMTC. 

In [19]:
for i in range(1,10):
    ind=i-1
    coords=norf_NMTCTracts['Coordinates'][ind]
    stype=norf_NMTCTracts['Type'][ind]
    pname=norf_NMTCTracts['2010 Census Tract'][ind]
    new_poly=Polygon(coords)
    popup=folium.Popup(str(pname))
    polylayer=folium.GeoJson(
        new_poly,
#        name=pname,
#        popup=(label)
    )#.add_child(folium.Popup(label))
#    print(label)
    popup.add_to(polylayer)
    polylayer.add_to(map_norf)
    
map_norf

Add in Norfolk address information

In [20]:
norf_Add=pd.read_csv('Spec_Add_Info.csv')
norf_Add.head()

,Full Address,Address Status,Parcel Centroid Lattitude,Parcel Centroid Longitude,State Plane X Coordinate,State Plane Y Coordinate,Census Block,Census Tract Number,Census Tract Name,New Georeferenced Column
0,918 TUNSTALL AVENUE,A,36.854052,-76.276917,12133298.12,3478079.247,2009,42.0,"CALVERT SQUARE, CHURCH STREET",POINT (-76.276752 36.853)
1,801 E VIRGINIA BEACH BOULEVARD,A,36.854052,-76.276917,12133298.12,3478079.247,2009,42.0,"CALVERT SQUARE, CHURCH STREET",POINT (-76.277925 36.855749)
2,904 E OLNEY ROAD,A,36.854052,-76.276917,12133298.12,3478079.247,2009,42.0,"CALVERT SQUARE, CHURCH STREET",POINT (-76.277763 36.853596)
3,847 E VIRGINIA BEACH BOULEVARD,A,36.854052,-76.276917,12133298.12,3478079.247,2009,42.0,"CALVERT SQUARE, CHURCH STREET",POINT (-76.27717 36.855425)
4,921 BAGNALL ROAD,A,36.854052,-76.276917,12133298.12,3478079.247,2009,42.0,"CALVERT SQUARE, CHURCH STREET",POINT (-76.276974 36.854711)


and clean it up...

In [21]:
norf_Add.drop(['State Plane X Coordinate','State Plane Y Coordinate','Census Block','New Georeferenced Column'],axis=1,inplace=True)
norf_Add.head()

,Full Address,Address Status,Parcel Centroid Lattitude,Parcel Centroid Longitude,Census Tract Number,Census Tract Name
0,918 TUNSTALL AVENUE,A,36.854052,-76.276917,42.0,"CALVERT SQUARE, CHURCH STREET"
1,801 E VIRGINIA BEACH BOULEVARD,A,36.854052,-76.276917,42.0,"CALVERT SQUARE, CHURCH STREET"
2,904 E OLNEY ROAD,A,36.854052,-76.276917,42.0,"CALVERT SQUARE, CHURCH STREET"
3,847 E VIRGINIA BEACH BOULEVARD,A,36.854052,-76.276917,42.0,"CALVERT SQUARE, CHURCH STREET"
4,921 BAGNALL ROAD,A,36.854052,-76.276917,42.0,"CALVERT SQUARE, CHURCH STREET"


In [22]:
norf_Add.dtypes

Full Address                  object
Address Status                object
Parcel Centroid Lattitude    float64
Parcel Centroid Longitude    float64
Census Tract Number          float64
Census Tract Name             object
dtype: object

In [23]:
norf_Add2=norf_Add
norf_Add['Census Tract Number']=norf_Add['Census Tract Number'].astype('object')

In [24]:
norf_Add['Census Tract Number'].value_counts()

29.00    1772
3.00     1015
26.00     980
49.00     893
66.04     872
40.02     829
44.00     729
47.00     555
42.00     541
Name: Census Tract Number, dtype: int64

In [25]:
norf_Add.drop(norf_Add[norf_Add['Full Address']=="NO ADDRESS"].index, inplace = True)

In [26]:
norf_Add.shape

(7798, 6)

Collect information on grocery stores and supermarkets from foursquare.com. This was a bit of a problem because using the search or explore function returned more than just the city of Norfolk. The info from these other cities ate into my 50 location max. In the end, I used Google maps, to chart the center of the city and the radius of the city of Norfolk.

In [27]:
CLIENT_ID = 'RQAZZWMEAEE0MNNHTRGZA55KKFA0VLWPXF0HDBBIZZLKWX0J' # your Foursquare ID
CLIENT_SECRET = 'ELEBUPLGHNG3ANIKXFH1M40Y015B4KFFD3IGTSIV2O2PETMR' # your Foursquare Secret
VERSION = '20200610' # Foursquare API version
ACCESS_TOKEN = 'MWQW1FCJIQNAG4QHD0DM4V5VD0T5GV3YKJOHXPM5ZVUZNBPD'
NEARTO = "Norfolk"
GEOID = '72057594042704158'
CATEGORY = '52f2ab2ebcbc57f1066b8b46,4bf58dd8d48988d118951735'
LIMIT = 200
radius = 7645
cen_lat = '36.891759'
cen_lon = '-76.256897'
#url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,ACCESS_TOKEN,VERSION,latitude,longitude,radius,LIMIT)
#url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&&v={}&near={}&categoryId={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,ACCESS_TOKEN,VERSION,NEARTO,CATEGORY,LIMIT)
#https://foursquare.com/explore?mode=url&near=Norfolk%2C%20VA%2C%20United%20States&nearGeoId=72057594042704158&q=grocery%20store
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,ACCESS_TOKEN,VERSION,cen_lat,cen_lon,CATEGORY,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RQAZZWMEAEE0MNNHTRGZA55KKFA0VLWPXF0HDBBIZZLKWX0J&client_secret=ELEBUPLGHNG3ANIKXFH1M40Y015B4KFFD3IGTSIV2O2PETMR&oauth_token=MWQW1FCJIQNAG4QHD0DM4V5VD0T5GV3YKJOHXPM5ZVUZNBPD&v=20200610&ll=36.891759,-76.256897&categoryId=52f2ab2ebcbc57f1066b8b46,4bf58dd8d48988d118951735&radius=7645&limit=200'

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
results=requests.get(url).json()

In [30]:
venues = results['response']['venues']
venues[0]    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name','location.lat', 'location.lng', 'location.city']
norfolk_grocery =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#norfolk_grocery['venue.categories'] = norfolk_grocery.apply(get_category_type, axis=1)

# clean columns
norfolk_grocery.columns = [col.split(".")[-1] for col in norfolk_grocery.columns]

norfolk_grocery.head(50)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,lat,lng,city
0,Food Lion Grocery Store,36.885440,-76.260088,Norfolk
1,ALDI,36.870453,-76.294155,Norfolk
2,Harris Teeter,36.954771,-76.253407,Norfolk
3,ALDI,36.858307,-76.209106,Norfolk
4,Murry Food Mart,36.855861,-76.195076,Norfolk
5,The Market,36.851804,-76.290078,Norfolk
6,Save-A-Lot,36.916141,-76.219153,Norfolk
7,Walmart Grocery Pickup and Delivery,36.912391,-76.252520,Norfolk
8,Walmart Grocery Pickup and Delivery,36.862153,-76.210130,Norfolk
9,Halal Grocery,36.854214,-76.194470,Norfolk


Clean up data from foursquare.com.

In [31]:
norfolk_grocery['city'].value_counts()

Norfolk           43
Virginia Beach     4
Chesapeake         1
Name: city, dtype: int64

In [32]:
#Cleaning up the data to remove convenience stores, and specialty food stores that do not fit the necessary 
#criteria. Also, stripping other cities from the data.
norfolk_grocery=norfolk_grocery[norfolk_grocery['city']=='Norfolk']
norfolk_grocery=norfolk_grocery[norfolk_grocery['name']!='Murry Food Mart']
norfolk_grocery=norfolk_grocery[~(norfolk_grocery.name.isin(["Murry Food Mart", "The Market","Halal Grocery","Westside Produce & Provisions",
                                                            "Tokyo Market","Nikko's Tindahan","Brahmani Indian Asian Market",
                                                            "Freddie's Seafood And Oriental Store","Fresh Pride","Mercy's Restaurant",
                                                            "Taj Bazar","P.O.D. Market - Quad","P.O.D. Market - Village","Shady Grove Marketplace"]))]


norfolk_grocery.shape

(27, 4)

In [33]:
norfolk_grocery.head(27)

,name,lat,lng,city
0,Food Lion Grocery Store,36.885440,-76.260088,Norfolk
1,ALDI,36.870453,-76.294155,Norfolk
2,Harris Teeter,36.954771,-76.253407,Norfolk
3,ALDI,36.858307,-76.209106,Norfolk
6,Save-A-Lot,36.916141,-76.219153,Norfolk
7,Walmart Grocery Pickup and Delivery,36.912391,-76.252520,Norfolk
8,Walmart Grocery Pickup and Delivery,36.862153,-76.210130,Norfolk
14,Save A Lot,36.853318,-76.278414,Norfolk
24,Walmart Supercenter,36.912225,-76.253625,Norfolk
26,The Fresh Market,36.872112,-76.298417,Norfolk


In [34]:
norfolk_grocery.reset_index(drop=True,inplace=True)

In [35]:
norfolk_grocery.head()

,name,lat,lng,city
0,Food Lion Grocery Store,36.885440,-76.260088,Norfolk
1,ALDI,36.870453,-76.294155,Norfolk
2,Harris Teeter,36.954771,-76.253407,Norfolk
3,ALDI,36.858307,-76.209106,Norfolk
4,Save-A-Lot,36.916141,-76.219153,Norfolk


Map grocery store info on top of NMTC tract map.

In [36]:
for lat, lon, name in zip(norfolk_grocery['lat'], norfolk_grocery['lng'], norfolk_grocery['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill_opacity=0.7).add_to(map_norf)
map_norf

If we draw a one mile circle around each of the grocery stores above we can see how much they overlap the NMTC tracts.

In [37]:
for lat, lon, name in zip(norfolk_grocery['lat'], norfolk_grocery['lng'], norfolk_grocery['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=1610,
        popup=label,
        color='red',
        fill_opacity=0.7).add_to(map_norf)
map_norf

There are 4 tracts that have areas that fall outside of the 1 mile radius of grocery stores. They are tract 03, 26, 29, and 47. 03 is a non-residential area around the airport. Let's focus on the other three tract numbers, so we can explore more thoroughly.

In [38]:
map_expnorf = folium.Map(location=[latitude, longitude], zoom_start=11)

In [39]:
norf_expNMTC = norf_NMTCTracts[(norf_NMTCTracts['2010 Census Tract'].isin([51710002600,51710004700,51710002900]))]
norf_expNMTC.reset_index(drop=True,inplace=True)

In [40]:
for i in range(1,4):
    ind=i-1
    coords=norf_expNMTC['Coordinates'][ind]
    stype=norf_expNMTC['Type'][ind]
    pname=norf_expNMTC['2010 Census Tract'][ind]
    new_poly=Polygon(coords)
    popup=folium.Popup(str(pname))
    polylayer=folium.GeoJson(
        new_poly,
#        name=pname,
        popup=(label)
    )#.add_child(folium.Popup(label))
#    print(label)
    popup.add_to(polylayer)
    polylayer.add_to(map_expnorf)
    
map_expnorf

In [41]:
norf_Add_lim = norf_Add[(norf_Add['Census Tract Number'].isin([26,47,29]))]

In [42]:
norf_Add_lim['Census Tract Number'].value_counts()

29.0    1710
26.0     894
47.0     494
Name: Census Tract Number, dtype: int64

In [43]:
norf_Add_lim.reset_index(drop=True,inplace=True)
norf_Add_lim.head()

,Full Address,Address Status,Parcel Centroid Lattitude,Parcel Centroid Longitude,Census Tract Number,Census Tract Name
0,891 GYM ROAD,A,36.848263,-76.262302,47,"BRAMBLETON (SOUTH), NSU"
1,427 37TH STREET,A,36.879359,-76.287867,29,PARK PLACE (EAST)
2,790 51ST STREET,A,36.889569,-76.294402,26,NORTH COLLEY
3,2111 PRESIDENTIAL PARKWAY,A,36.847669,-76.266957,47,"BRAMBLETON (SOUTH), NSU"
4,2150 PRESIDENTIAL PARKWAY,A,36.847713,-76.267405,47,"BRAMBLETON (SOUTH), NSU"


In [44]:
for lat, lon, name in zip(norfolk_grocery['lat'], norfolk_grocery['lng'], norfolk_grocery['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=1610,
        popup=label,
        color='red',
#        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_expnorf)

for addlat, addlon in zip(norf_Add_lim['Parcel Centroid Lattitude'], norf_Add_lim['Parcel Centroid Longitude']):
    folium.Marker(
        [addlat, addlon]).add_to(map_expnorf)

map_expnorf

Now let's calculate the distance from each address to its closest grocery store.

In [45]:
from geopy import distance

In [46]:
#Make field names more friendly
norf_Add_lim.rename(columns = {'Parcel Centroid Lattitude':'Latitude','Parcel Centroid Longitude':'Longitude'}, inplace=True)
norf_Add_lim.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Full Address,Address Status,Latitude,Longitude,Census Tract Number,Census Tract Name
0,891 GYM ROAD,A,36.848263,-76.262302,47,"BRAMBLETON (SOUTH), NSU"
1,427 37TH STREET,A,36.879359,-76.287867,29,PARK PLACE (EAST)
2,790 51ST STREET,A,36.889569,-76.294402,26,NORTH COLLEY
3,2111 PRESIDENTIAL PARKWAY,A,36.847669,-76.266957,47,"BRAMBLETON (SOUTH), NSU"
4,2150 PRESIDENTIAL PARKWAY,A,36.847713,-76.267405,47,"BRAMBLETON (SOUTH), NSU"


In [47]:
norf_Add_lim2=norf_Add_lim
norf_Add_lim['Grocery Store Distance']=""
for ind, res_lat, res_lon in zip(norf_Add_lim.index, norf_Add_lim['Latitude'],norf_Add_lim['Longitude']):
    res_loc = (res_lat, res_lon)
    short_dis=50

    for lat, lon in zip(norfolk_grocery['lat'], norfolk_grocery['lng']):
        groc_loc = (lat, lon)
        groc_dis = (distance.distance(res_loc, groc_loc).miles)
        if(short_dis>groc_dis):
            short_dis=groc_dis
    norf_Add_lim['Grocery Store Distance'][ind]=short_dis

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [48]:
norf_Add_lim.head()

,Full Address,Address Status,Latitude,Longitude,Census Tract Number,Census Tract Name,Grocery Store Distance
0,891 GYM ROAD,A,36.848263,-76.262302,47,"BRAMBLETON (SOUTH), NSU",0.958535
1,427 37TH STREET,A,36.879359,-76.287867,29,PARK PLACE (EAST),0.617295
2,790 51ST STREET,A,36.889569,-76.294402,26,NORTH COLLEY,1.10445
3,2111 PRESIDENTIAL PARKWAY,A,36.847669,-76.266957,47,"BRAMBLETON (SOUTH), NSU",0.744909
4,2150 PRESIDENTIAL PARKWAY,A,36.847713,-76.267405,47,"BRAMBLETON (SOUTH), NSU",0.722219


Create a dataframe with the new information.

In [49]:
column_names=['Tract Name','Tract Number', 'Total Residences', 'No. Outside Radius']
norf_info = pd.DataFrame(columns=column_names)

In [50]:
norf_info.head()

,Tract Name,Tract Number,Total Residences,No. Outside Radius


In [51]:
norf_info['Tract Number']=(47,26,29)
for ind, tract in zip(norf_info.index, norf_info['Tract Number']):
    total=len(norf_Add_lim[norf_Add_lim['Census Tract Number']==tract])
    outside=len(norf_Add_lim[(norf_Add_lim['Census Tract Number']==tract)&(norf_Add_lim['Grocery Store Distance']>1)])
    norf_info['Total Residences'][ind]=total
    norf_info['No. Outside Radius'][ind]=outside
#    if(norf_Add_lim['Census Tract Number']==tract):
#        print(norf_Add_lim['Census Tract Name'])
#    norf_info['Tract Name']=(norf_Add_lim[norf_Add['Census Tract Name']])
    
#norf_Add_lim['Census Tract Name'].unique()
#norf_info['Tract Name']=norf_Add_lim.loc[norf_Add_lim['Census Tract Number']=]
#df2['Population'] = df2.apply(lambda x: df1.loc[x['Year'] == df1['Year'], 
#                                                x['State']].reset_index(drop=True), axis=1)




/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
for i in range(1,4):
    j=i-1
    tract=norf_info['Tract Number'][j]
    indexPos=((norf_Add_lim['Census Tract Number']).tolist()).index(tract)
    name=norf_Add_lim['Census Tract Name'][indexPos]
    norf_info['Tract Name'][j]=name


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
norf_info.head()

,Tract Name,Tract Number,Total Residences,No. Outside Radius
0,"BRAMBLETON (SOUTH), NSU",47,494,19
1,NORTH COLLEY,26,894,206
2,PARK PLACE (EAST),29,1710,154


In [54]:
norf_info2=norf_info
norf_info['Percentage']= norf_info.apply(lambda row: row['No. Outside Radius']/row['Total Residences']*100, axis=1) 
norf_info.head()

,Tract Name,Tract Number,Total Residences,No. Outside Radius,Percentage
0,"BRAMBLETON (SOUTH), NSU",47,494,19,3.846154
1,NORTH COLLEY,26,894,206,23.042506
2,PARK PLACE (EAST),29,1710,154,9.005848


## CONCLUSION

As we can see from the above data frame, none of the neighborhoods in Norfolk fit into the 33% requirement of the food desert categorization. 

Generally speaking it looks like Norfolk, VA is serving its populace relatively well in terms of access to fresh produce. 

One limitation of this data is that it only deals with residential <b>addresses</b>. The food desert definition is actually dealing with individual residents. This means that this data would need to be expanded to include more contemporary census data about the number of individuals that live at each address. That is beyond the scope of this report.

However, particular attention should be paid to the North Colley area, as there are 206 addresses that fall outside of the 1 mile grocery store radius. If an average of 2.5 people live at each address, than North Colley would reach the 500 resident threshhold necessary to classify it as a food desert.


<i>This report was developed in partial fulfillment of requirements for the capstone project of the IBM Applied Data Science Certification.</i> All data was collected and shaped by Charles Anthony Robinson.